In [ ]:
%load_ext autoreload
%autoreload 2

bhp_project = 15789 # The project in BHP
gv_project = 8381499 # The project id in GEOVISTORY
pk_gv_symogih = 6857901 # The pk_project of symogih on GEOVISTORY

env = 'prod'
execute = True
metadata_str = 'bhp-add-entity-to-project-Siprojuris'
import_manner = 'one-shot'

import os
import pandas as pd
import numpy as np
from datetime import datetime
import duckdb
import plotly.express as px

import geovpylib.analysis as a
import geovpylib.database as db
import geovpylib.graphs as graphs
import geovpylib.pks as pks
import geovpylib.recordlinkage as rl
import geovpylib.sparql as sparql
import geovpylib.utils as u

eta = u.Eta()

# Add entities to project

The goal here is to add a list of entities that were in a particular BHP project, to the equivalent Geovistory project.

## Fetch Geovistory SYMOGIH data

In [ ]:
db.connect_geovistory(env, pk_gv_symogih, False, skip_protection=True)

symogih_entities_gv = db.query(f"""
    select
        r.pk_entity as pk_gv,
        a3.string as uri        
    from information.resource r 
    inner join projects.info_proj_rel ipr on ipr.fk_entity = r.pk_entity and ipr.fk_project = {pk_gv_symogih} and ipr.is_in_project = true
    -- URI
    inner join information.statement s1 on s1.fk_subject_info = r.pk_entity and s1.fk_property = {pks.properties.entity_sameAsURI_URI}
    inner join projects.info_proj_rel ipr1 on ipr1.fk_entity = s1.pk_entity and ipr1.fk_project = {pk_gv_symogih} and ipr1.is_in_project = true
    inner join information.statement s2 on s2.fk_subject_info = s1.fk_object_info and s2.fk_property = {pks.properties.appe_hasValue_string}
    inner join information.appellation a3 on a3.pk_entity = s2.fk_object_info
""")

# Only symogih URIs
symogih_entities_gv = symogih_entities_gv[symogih_entities_gv['uri'].str.contains('symogih.org')]

# Extract PK
symogih_entities_gv['pk_bhp'] = symogih_entities_gv['uri'].str.replace('http://symogih.org/resource/', '', regex=False)
symogih_entities_gv.drop(columns=['uri'], inplace=True)

a.infos(symogih_entities_gv)

db.disconnect()

# 12s

## Fetch BHP entities

In [ ]:
db.connect_external(os.environ.get('YELLOW_BHP'))

symogih_entities_bhp = db.query(f"""
    select 
        fk_collective_actor as pk_bhp_project, fk_object as pk_bhp
    from bhp.associate_project ap
    where ap.fk_collective_actor = {bhp_project}
""")

symogih_entities_bhp = symogih_entities_bhp[['pk_bhp']]

a.infos(symogih_entities_bhp)

db.disconnect()

# 2s

## Filter only the wanted

In [ ]:
entities = symogih_entities_gv.merge(symogih_entities_bhp, how='inner')['pk_gv'].tolist()

print('Entity number:', len(entities))

## Get all information we have about those entities in Geovistory Symogih project

In [ ]:
db.connect_geovistory(env, -1, False, skip_protection=True)


# Get the whole graph ids of those entities (with rules; eg only incoming properties for TeEn)
peits, teens, stmts = graphs.get_graph(entities, pk_gv_symogih)

# Make the graph explicit
# the_graph = db.explicit_statements(stmts)
# print('Explicit graph:')
# display(the_graph)

# Make a visual graph out of the explicit statements  
# graphs.show(the_graph, 'test.html')

db.disconnect()

# 55s

## Add those information to the project

In [ ]:
db.connect_geovistory(env, gv_project, execute)

### Before import

In [ ]:
ipr_before = db.query(f'select pk_entity from projects.info_proj_rel where fk_project = {gv_project} and is_in_project = true')

print('Number of information before import:', len(ipr_before))

In [ ]:
db.info_proj_rels.create(entities)
db.info_proj_rels.create(peits)
db.info_proj_rels.create(teens)
db.info_proj_rels.create(stmts)

In [ ]:
ipr_after = db.query(f'select pk_entity from projects.info_proj_rel where fk_project = {gv_project} and is_in_project = true')

print('Number of information after import:', len(ipr_after))